# A Macroscopically Estimating Model with Endogeneity Considered for Lane-mean Speeds and Speed Deviations (2lanes)

## 1 Structuring Data

### 1.1 Import Data

In [4]:
import numpy as np
import pandas as pd
import xlrd
from linearmodels import IV3SLS
import datetime
import math
import os
from dateutil.relativedelta import * # for incement of month

In [5]:
# filelist = []
# file_dir = 'G:\\Modeling Endogeneity\\Data'

# for root, dirs, files in os.walk(file_dir):
#     filelist = files

# file_200m_2lanes = [file for file in filelist if '200m_2lanes' in file]

In [6]:
data_up_fs = pd.DataFrame()
data_do_fs = pd.DataFrame()
data_up_o = pd.DataFrame()
data_up_tftp = pd.DataFrame()
data_up_vmtvht = pd.DataFrame()

for num in range(1,12): # 400m_2lanes dataset lacks the data of Jan, 2017, so the range is from Feb to Dec
    monnum = num + 1
    # flow,speed
    file_up_fs = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_up_fs_' + str(monnum) + '.xlsx'
    file_do_fs = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_do_fs_' + str(monnum) + '.xlsx'
    # occupancy
    file_up_o = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_up_o_' + str(monnum) + '.xlsx'
    file_do_o = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_do_o_' + str(monnum) + '.xlsx'
    # truck flow, truck percentage
    file_up_tftp = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_up_tftp_' + str(monnum) + '.xlsx'
    # file_do_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_tftp_1.xlsx'
    # truck vmt, trauck vht
    file_up_vmtvht = 'G:\\Modeling Endogeneity\\Data\\400m_2lanes_up_vmtvht_' + str(monnum) + '.xlsx'
    # file_do_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_vmtvht_1.xlsx'

    data_up_fs_t = pd.read_excel(file_up_fs)
    data_do_fs_t = pd.read_excel(file_do_fs)
    data_up_o_t = pd.read_excel(file_up_o)
    # data_do_o = pd.read_excel(file_do_o)
    data_up_tftp_t = pd.read_excel(file_up_tftp)
    # data_do_tftp = pd.read_excel(file_do_tftp)
    data_up_vmtvht_t = pd.read_excel(file_up_vmtvht)
    # data_do_vmtvht = pd.read_excel(file_do_vmtvht)
    data_up_fs = data_up_fs.append(data_up_fs_t, sort=True)
    data_do_fs = data_do_fs.append(data_do_fs_t, sort=True)
    data_up_o = data_up_o.append(data_up_o_t, sort=True)
    data_up_tftp = data_up_tftp.append(data_up_tftp_t, sort=True)
    data_up_vmtvht = data_up_vmtvht.append(data_up_vmtvht_t, sort=True)

In [7]:
data_up_vmtvht.head() # 3/12/2017 凌晨2:00-2:55的数据缺失所以少了12条数据

,# Lane Points,% Observed,5 Minutes,Lane 1 Truck VHT (Veh-Hours),Lane 1 Truck VMT (Veh-Miles),Lane 2 Truck VHT (Veh-Hours),Lane 2 Truck VMT (Veh-Miles),Truck VHT (Veh-Hours),Truck VMT (Veh-Miles)
0,2,100,2/08/2017 0:00,0,0,0.0,0.0,0.0,0.0
1,2,100,2/08/2017 0:05,0,0,0.0,0.0,0.0,0.0
2,2,100,2/08/2017 0:10,0,0,0.0,0.0,0.0,0.0
3,2,100,2/08/2017 0:15,0,0,0.0,0.0,0.0,0.0
4,2,100,2/08/2017 0:20,0,0,0.0,0.0,0.0,0.0


In [8]:
data_do_fs.describe()

,# Lane Points,% Observed,Flow (Veh/5 Minutes),Lane 1 Flow (Veh/5 Minutes),Lane 1 Speed (mph),Lane 2 Flow (Veh/5 Minutes),Lane 2 Speed (mph),Speed (mph)
count,22164.0,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000
mean,2.0,99.553330,125.852554,59.272559,64.698407,66.579995,59.416247,62.266554
std,0.0,6.668547,82.232806,35.570201,12.522596,47.621900,13.048514,12.919058
min,2.0,0.000000,0.000000,0.000000,5.900000,0.000000,4.700000,5.300000
25%,2.0,100.000000,41.000000,24.000000,65.200000,17.000000,61.300000,63.100000
50%,2.0,100.000000,143.000000,67.000000,67.800000,74.000000,63.400000,65.400000
75%,2.0,100.000000,197.000000,90.000000,71.000000,108.000000,65.000000,68.500000
max,2.0,100.000000,340.000000,158.000000,79.600000,203.000000,71.600000,74.500000


In [9]:
# Caution: use dataframe.head() to observe the name and the position of every column
data_up = data_up_fs
data_up = data_up.iloc[:,2:9]
data_up.columns = ['t','f','f1','s1','f2','s2','s']
data_up[['o1','o2']] = data_up_o.iloc[:,[3,4]]
data_up[['tf1','tp1','tf2','tp2']] = data_up_tftp.iloc[:,[3,4,5,6]]
data_up[['vht1','vmt1','vht2','vmt2']] = data_up_vmtvht.iloc[:,[3,4,5,6]]
data_up[['s_do1','s_do2']] = data_do_fs.iloc[:,[5,7]]
data_up = data_up.reset_index(drop=True)
data_up['t'] = pd.to_datetime(data_up['t'])

In [10]:
data_up.describe()

,f,f1,s1,f2,s2,s,o1,o2,tf1,tp1,tf2,tp2,vht1,vmt1,vht2,vmt2,s_do1,s_do2
count,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.000000,22164.0,22164.0,22164.000000,22164.000000,22164.0,22164.0,22164.000000,22164.000000,22164.000000,22164.000000
mean,109.689136,49.548502,65.927378,60.140633,60.046192,63.086591,5.328104,6.209096,0.0,0.0,0.250677,0.326511,0.0,0.0,0.006483,0.089289,64.698407,59.416247
std,72.492778,29.241880,11.834407,44.120814,11.389389,11.733434,5.757060,7.232811,0.0,0.0,1.625570,2.201450,0.0,0.0,0.051138,0.574866,12.522596,13.048514
min,0.000000,0.000000,3.400000,0.000000,3.500000,3.900000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,5.900000,4.700000
25%,36.000000,21.000000,66.600000,15.000000,60.600000,63.300000,1.800000,1.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,65.200000,61.300000
50%,119.000000,54.000000,69.700000,63.000000,63.800000,66.400000,4.800000,5.400000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,67.800000,63.400000
75%,171.000000,74.000000,71.100000,98.000000,65.000000,68.500000,6.600000,8.300000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,71.000000,65.000000
max,329.000000,138.000000,81.200000,195.000000,74.200000,74.200000,68.900000,69.000000,0.0,0.0,41.000000,49.400000,0.0,0.0,1.400000,14.500000,79.600000,71.600000


###  1.2 Delete Outlier

In [11]:
data_up['t'] = pd.to_datetime(data_up['t'])
data = pd.DataFrame()
for month in range(11):
    
    for day in range(7):
        begin = data_up['t'][0] + datetime.timedelta(days=day) + relativedelta(months=month)
        end = data_up['t'][0] + datetime.timedelta(days=day+1) + relativedelta(months=month)
        interval = (data_up['t']>begin) & (data_up['t']<end)
        data_up_t = data_up[interval]

        upper_q_s1 = data_up_t['s1'].quantile(0.75)
        lower_q_s1 = data_up_t['s1'].quantile(0.25)
        IQR_s1 = upper_q_s1 - lower_q_s1
        upper_q_s2 = data_up_t['s2'].quantile(0.75)
        lower_q_s2 = data_up_t['s2'].quantile(0.25)
        IQR_s2 = upper_q_s2 - lower_q_s2
        data_up_t = data_up_t[((lower_q_s1-1.5*IQR_s1) <= data_up_t['s1'])& (data_up_t['s1'] <= (upper_q_s1+1.5*IQR_s1))]
        data_up_t = data_up_t[((lower_q_s2-1.5*IQR_s2) <= data_up_t['s2'])& (data_up_t['s2'] <= (upper_q_s2+1.5*IQR_s2))]

        data = data.append(data_up_t, sort = True)
data_up = data
data_up.describe()

,f,f1,f2,o1,o2,s,s1,s2,s_do1,s_do2,tf1,tf2,tp1,tp2,vht1,vht2,vmt1,vmt2
count,20232.00000,20232.000000,20232.000000,20232.000000,20232.000000,20232.000000,20232.000000,20232.000000,20232.000000,20232.000000,20232.0,20232.000000,20232.0,20232.000000,20232.0,20232.000000,20232.0,20232.000000
mean,105.75514,47.771748,57.983393,4.121451,4.678801,65.926626,68.765342,62.827234,67.714675,62.608210,0.0,0.022193,0.0,0.023166,0.0,0.000020,0.0,0.008284
std,72.53217,29.050264,44.296177,2.558174,3.644098,3.629151,3.534771,3.257062,5.171372,5.359815,0.0,0.225750,0.0,0.231853,0.0,0.001406,0.0,0.082138
min,0.00000,0.000000,0.000000,0.000000,0.000000,40.000000,40.900000,37.800000,15.900000,9.100000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
25%,31.00000,19.000000,12.000000,1.600000,1.000000,64.100000,67.500000,61.600000,65.700000,61.700000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
50%,108.00000,51.000000,56.000000,4.300000,4.500000,66.800000,70.100000,64.100000,68.600000,64.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
75%,171.00000,74.000000,97.000000,6.400000,7.800000,68.600000,71.100000,65.000000,71.000000,65.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
max,303.00000,127.000000,191.000000,21.000000,25.600000,73.300000,78.600000,74.200000,76.400000,68.800000,0.0,9.000000,0.0,12.500000,0.0,0.100000,0.0,3.200000


In [12]:
data_up.head()

,f,f1,f2,o1,o2,s,s1,s2,s_do1,s_do2,t,tf1,tf2,tp1,tp2,vht1,vht2,vmt1,vmt2
1,7,4,3,0.4,0.3,68.4,71.0,64.9,71.1,65.1,2017-02-08 00:05:00,0,0,0,0.0,0,0.0,0,0.0
2,15,11,4,1.0,0.4,69.2,70.8,64.8,71.0,65.0,2017-02-08 00:10:00,0,0,0,0.0,0,0.0,0,0.0
3,10,7,3,0.6,0.2,69.1,70.9,64.9,71.0,65.0,2017-02-08 00:15:00,0,0,0,0.0,0,0.0,0,0.0
4,10,9,1,0.8,0.1,70.4,71.0,64.8,71.0,65.1,2017-02-08 00:20:00,0,0,0,0.0,0,0.0,0,0.0
5,9,6,3,0.5,0.2,69.0,71.0,65.1,71.0,65.1,2017-02-08 00:25:00,0,0,0,0.0,0,0.0,0,0.0


### 1.3 Construct Variables

In [13]:
data_up = data_up[(data_up['f1']!=0) & (data_up['f2']!=0)]
data_up.describe()

,f,f1,f2,o1,o2,s,s1,s2,s_do1,s_do2,tf1,tf2,tp1,tp2,vht1,vht2,vmt1,vmt2
count,19865.000000,19865.000000,19865.00000,19865.000000,19865.000000,19865.000000,19865.000000,19865.000000,19865.000000,19865.000000,19865.0,19865.000000,19865.0,19865.000000,19865.0,19865.000000,19865.0,19865.000000
mean,107.617669,48.565870,59.05180,4.190103,4.764999,65.838203,68.721601,62.786579,67.653063,62.563332,0.0,0.022603,0.0,0.023594,0.0,0.000020,0.0,0.008437
std,71.879928,28.715639,43.99385,2.530664,3.621473,3.597685,3.552383,3.273001,5.198444,5.398721,0.0,0.227806,0.0,0.233963,0.0,0.001419,0.0,0.082886
min,2.000000,1.000000,1.00000,0.100000,0.000000,40.000000,40.900000,37.800000,15.900000,9.100000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
25%,35.000000,20.000000,14.00000,1.700000,1.100000,64.100000,67.500000,61.500000,65.700000,61.700000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
50%,113.000000,52.000000,59.00000,4.500000,4.700000,66.700000,70.000000,64.000000,68.400000,63.800000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
75%,171.000000,74.000000,98.00000,6.400000,7.800000,68.600000,71.100000,65.000000,71.000000,65.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
max,303.000000,127.000000,191.00000,21.000000,25.600000,73.300000,78.600000,74.200000,76.400000,68.800000,0.0,9.000000,0.0,12.500000,0.0,0.100000,0.0,3.200000


In [14]:
lane1list = ['t']+[col for col in data_up.columns if '1' in col]
data1 = data_up[lane1list]
lane2list = ['t']+[col for col in data_up.columns if '2' in col]
data2 = data_up[lane2list]
data1.head()

,t,f1,o1,s1,s_do1,tf1,tp1,vht1,vmt1
1,2017-02-08 00:05:00,4,0.4,71.0,71.1,0,0,0,0
2,2017-02-08 00:10:00,11,1.0,70.8,71.0,0,0,0,0
3,2017-02-08 00:15:00,7,0.6,70.9,71.0,0,0,0,0
4,2017-02-08 00:20:00,9,0.8,71.0,71.0,0,0,0,0
5,2017-02-08 00:25:00,6,0.5,71.0,71.0,0,0,0,0


In [15]:

variables = ['t','f','o','s','s_do','tf','tp','vht','vmt']
data1.columns = variables
data2.columns = variables
data1.loc[:,'s2'] = data2.loc[:,'s']
data2.loc[:,'s1'] = data1.loc[:,'s']
data1.head()

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,t,f,o,s,s_do,tf,tp,vht,vmt,s2
1,2017-02-08 00:05:00,4,0.4,71.0,71.1,0,0,0,0,64.9
2,2017-02-08 00:10:00,11,1.0,70.8,71.0,0,0,0,0,64.8
3,2017-02-08 00:15:00,7,0.6,70.9,71.0,0,0,0,0,64.9
4,2017-02-08 00:20:00,9,0.8,71.0,71.0,0,0,0,0,64.8
5,2017-02-08 00:25:00,6,0.5,71.0,71.0,0,0,0,0,65.1


In [16]:
# use dummy variablesS
for lanedata in [data1, data2]:
    # ln(speed)
    lanedata.loc[:,'lns'] = lanedata['s'].map(lambda x: math.log(x))
    # truck speed
    lanedata.loc[:,'ts'] = 999
    lanedata.loc[lanedata['vht']!=0, 'ts'] =  lanedata['vmt']/lanedata['vht']
#     lanedata['ts'] = lanedata.apply(lambda x: 999.0 if x['vht'] == 0 else x['vmt']/x['vht'], axis = 1) 
#     '''axis : {0 or ‘index’, 1 or ‘columns’}, default 0
#         Axis along which the function is applied:
#         0 or ‘index’: apply function to each column.
#         1 or ‘columns’: apply function to each row.'''
    # traffic flow indicator and high truck flow indicator
    lanedata.loc[:,'fi'] = 0
    lanedata.loc[:,'htfi'] = 0
    lanedata.loc[lanedata['f']<75, 'fi'] = 1
    lanedata.loc[lanedata['tf']>100, 'htfi'] = 1
    # relative truck percentage indicator
    lanedata.loc[:,'tpia'] = 0
    lanedata.loc[:,'tpib'] = 0
    lanedata.loc[(lanedata['tp']>0.6)&(lanedata['f']<50), 'tpia'] = 1
    lanedata.loc[(lanedata['tp']<=0.2)&(lanedata['f']>200), 'tpib'] = 1
    # seasonal indicator (fall will be reference)
    lanedata.loc[:,'seasona'] = 0
    lanedata.loc[:,'seasonb'] = 0
    lanedata.loc[:,'seasonc'] = 0
    lanedata.loc[(2<lanedata['t'].dt.month) & (lanedata['t'].dt.month<6), 'seasona'] = 1 # Spring
    lanedata.loc[(5<lanedata['t'].dt.month) & (lanedata['t'].dt.month<9), 'seasonb'] = 1
    lanedata.loc[(8<lanedata['t'].dt.month) & (lanedata['t'].dt.month<12), 'seasonc'] = 1
    # time-of-week indicator
    lanedata.loc[:,'weeka'] = 0
    lanedata.loc[:,'weekb'] = 0
    lanedata.loc[:,'weekc'] = 0
    lanedata.loc[:,'weekd'] = 0
    lanedata.loc[:,'weeke'] = 0
    lanedata.loc[:,'weekf'] = 0
    lanedata.loc[lanedata['t'].dt.weekday==0, 'weeka'] = 1 # Monday
    lanedata.loc[lanedata['t'].dt.weekday==1, 'weekb'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==2, 'weekc'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==3, 'weekd'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==4, 'weeke'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==5, 'weekf'] = 1
    # time-of-day indicator
    lanedata['hour'] = lanedata['t'].dt.hour
    lanedata.loc[:,'toda'] = 0
    lanedata.loc[:,'todb'] = 0
    lanedata.loc[:,'todc'] = 0
    lanedata.loc[:,'todd'] = 0
    lanedata.loc[(0<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<6), 'toda'] = 1 # midnight to 6 am
    lanedata.loc[(7<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<9), 'todb'] = 1 # am peak
    lanedata.loc[(17<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<19), 'todc'] = 1 # pm peak
    lanedata.loc[(19<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<24), 'todd'] = 1 # night time
    
# ratio of flows in current lane to adjacent lanes
data1.loc[:,'fratio'] = data1['f']/data2['f']
data2.loc[:,'fratio'] = data2['f']/data1['f']

In [17]:
# data1['fratio'] = data1['f']/data2['f']
# data1 = data1.dropna()
data1.dtypes

t          datetime64[ns]
f                   int64
o                 float64
s                 float64
s_do              float64
tf                  int64
tp                  int64
vht                 int64
vmt                 int64
s2                float64
lns               float64
ts                float64
fi                  int64
htfi                int64
tpia                int64
tpib                int64
seasona             int64
seasonb             int64
seasonc             int64
weeka               int64
weekb               int64
weekc               int64
weekd               int64
weeke               int64
weekf               int64
hour                int64
toda                int64
todb                int64
todc                int64
todd                int64
fratio            float64
dtype: object

## 2 Prediction 

### 2.1 Error Examination

In [18]:
# data1[['f','fratio']] = data1[['f','fratio']].infer_objects()
# data2[['f','fratio']] = data2[['f','fratio']].infer_objects()
''' Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.
And the error come from other_var.
And after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'
are (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object 
to int and float, respectively.
'''

" Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.\nAnd the error come from other_var.\nAnd after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'\nare (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object \nto int and float, respectively.\n"

In [19]:
np.linalg.matrix_rank(data1[['o','f','ts','fi','fratio']])

5

In [20]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### 2.2 Predicting Used New Method (including downstream speed)

In [21]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_new = pd.DataFrame(columns=['MAE1','MAE2'])
speed_new1 = pd.DataFrame()
speed_new2 = pd.DataFrame()
for month in range(1,12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var+['s_do1']],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments+['s_do2']]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var+['s_do2']],
         'endog': data2_train[['s1']],
         'instruments': data1_train[instruments+['s_do1']]}
    equations = dict(u1=u1, u2=u2)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var+['s_do1']],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments+['s_do2']]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var+['s_do2']],
         'endog': data2_test[['s1']],
         'instruments': data1_test[instruments+['s_do1']]}
    equations_test = dict(u1=u1_test, u2=u2_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_new.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_new.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    speed_new1 = pd.concat([speed_new1,y_pred1])
    speed_new2 = pd.concat([speed_new2,y_pred2])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # 这个语句的结果只有一个月的数据

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self

### 2.3 Predicting Used Old Method (not including downstream speed)

In [22]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_old = pd.DataFrame(columns=['MAE1','MAE2'])
speed_old1 = pd.DataFrame()
speed_old2 = pd.DataFrame()
for month in range(1,12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var],
         'endog': data2_train[['s1']],
         'instruments': data1_train[instruments]}
    equations = dict(u1=u1, u2=u2)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var],
         'endog': data2_test[['s1']],
         'instruments': data1_test[instruments]}
    equations_test = dict(u1=u1_test, u2=u2_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_old.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_old.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    speed_old1 = pd.concat([speed_old1,y_pred1])
    speed_old2 = pd.concat([speed_old2,y_pred2])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # 这个语句的结果只有一个月的数据

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self

### 2.4 Result Exporting

In [23]:
# speed_new1.columns = ['new1']
# speed_new2.columns = ['new2']
# speed_old1.columns = ['old1']
# speed_old2.columns = ['old2']

In [27]:
speed = speed_new1
speed['new2'] = speed_new2
speed['old1'] = speed_old1
speed['old2'] = speed_old2
speed['observed1'] = data1['s']
speed['observed2'] = data2['s']
speed.rename(columns={0: 'new1'}, inplace=True)
speed.head()

,new1,new2,old1,old2,observed1,observed2
1,70.419990,65.544528,61.662871,73.442098,71.0,64.9
2,70.313956,65.350821,61.578159,73.217417,70.8,64.8
3,70.416284,65.450840,61.662871,73.329757,70.9,64.9
4,70.313956,65.544528,61.578159,73.442098,71.0,64.8
5,70.620940,65.544528,61.832294,73.442098,71.0,65.1


In [25]:
error_result = pd.DataFrame()
error_result['MAE_new1'] = error_result_new['MAE1']
error_result['MAE_new2'] = error_result_new['MAE2']
error_result['MAE_old1'] = error_result_old['MAE1']
error_result['MAE_old2'] = error_result_old['MAE2']
error_result

,MAE_new1,MAE_new2,MAE_old1,MAE_old2
1,68.9774,65.0786,60.6837,72.6127
2,70.5813,64.835,70.5329,64.9084
3,69.9207,63.6743,69.9482,63.6219
4,69.8333,63.6743,69.8511,63.6564
5,66.7131,60.9876,66.6464,60.9855
6,66.2501,58.255,66.3194,58.0828
7,66.1702,63.8941,65.9878,64.4865
8,68.1197,63.1191,68.397,63.0203
9,71.441,62.2305,70.9784,62.263
10,67.7923,63.593,67.9899,63.6415


In [26]:
speed.to_excel('G:\\Modeling Endogeneity\\Results\\400m_2lanes_speed.xlsx')
error_result.to_excel('G:\\Modeling Endogeneity\\Results\\400m_2lanes_error.xlsx')
data_up[['t','f1','f2']].to_excel('G:\\Modeling Endogeneity\\Results\\400m_2lanes_flow.xlsx')